In [ ]:
import re
import numpy as np
import tritongrpcclient
from PIL import Image
import time
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import os

In [ ]:
def Relcrop(image, b,factors=(0.35,0.2,0.15,0.15)):
    landmarks_top = b[1]
    landmarks_bottom = b[3]
    landmarks_left = b[0]
    landmarks_right = b[2]
    top = int(landmarks_top - factors[0] * (landmarks_bottom - landmarks_top))
    bottom = int(landmarks_bottom + factors[1] * (landmarks_bottom - landmarks_top))
    left = int(landmarks_left - factors[2] * (landmarks_right - landmarks_left))
    right = int(landmarks_right + factors[3] * (landmarks_right - landmarks_left))
    
    if bottom - top > right - left:
        left -= ((bottom - top) - (right - left)) // 2
        right = left + (bottom - top)
    else:
        top -= ((right - left) - (bottom - top)) // 2
        bottom = top + (right - left)
    image_crop = np.ones(
        (bottom - top + 1, right - left + 1, 3), np.uint8) * 255
    h, w = image.shape[:2]
    left_white = max(0, -left)
    left = max(0, left)
    right = min(right, w-1)
    right_white = left_white + (right-left)
    top_white = max(0, -top)
    top = max(0, top)
    bottom = min(bottom, h-1)
    bottom_white = top_white + (bottom - top)

    image_crop[top_white:bottom_white+1, left_white:right_white +
                1] = image[top:bottom+1, left:right+1].copy()
    return image_crop

In [ ]:
triton_client = tritongrpcclient.InferenceServerClient(url="216.48.178.109:8061", verbose=False)

In [ ]:
def tritonJob(inputPath,outputPath):
    triton_client = tritongrpcclient.InferenceServerClient(url="216.48.178.109:8061", verbose=False)
    inputs = []
    outputs = []
    input0_name = "INPUT__0"
    output0_name = "OUTPUT__0"
    img = Image.open(inputPath).convert('RGB')
    width, height = img.size
    hpercent  = (500 / float(img.size[1]))
    hsize = int((float(img.size[0]) * float(hpercent)))
    img = img.resize((hsize, 500), Image.ANTIALIAS)



    try:
        image_data = np.array(img).astype("uint8")
        image_data = np.expand_dims(image_data, axis = 0)
        inputs.append(tritongrpcclient.InferInput(input0_name, image_data.shape, "UINT8"))
        outputs.append(tritongrpcclient.InferRequestedOutput(output0_name))

        inputs[0].set_data_from_numpy(image_data)
        response  = triton_client.infer(model_name="retinaface_ensemble",
                                        inputs=inputs,
                                        outputs=outputs)

        result = response.get_response()
        bboxes = response.as_numpy(output0_name)
        bboxes = bboxes[0]
        # face_crop = Relcrop(np.array(img),bboxes[0],(0,0,0,0))
        face_crop = Relcrop(np.array(img),bboxes[0])
        Image.fromarray(face_crop).resize((256,256)).save(outputPath)
    except Exception as e:
        print(inputPath)
        error_message = re.findall('TritonModelException: ([a-zA-Z0-9 ]+)\n', str(e))
        print(error_message)
        # exit()
        return



In [ ]:
inputP = "./data/input/segTest"
outputN = inputP.split("/")[-1]
outputP = os.path.join("./data/output",outputN+"-output")
if not os.path.exists(outputP):
    os.makedirs(outputP)

In [ ]:
Parallel(n_jobs=50)(delayed(tritonJob)(os.path.join(inputP,imgN),os.path.join(outputP,imgN)) for imgN in tqdm(os.listdir(inputP)))

In [ ]:
# for imgN in os.listdir(inputDir):
#     inputPath = os.path.join(inputDir,imgN)
#     outputPath = os.path.join(outputDir,imgN)
#     tritonJob(inputPath,outputPath)
    
